In [232]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [282]:
user_days = pd.read_csv('data/features/user_days.csv')
activity = pd.read_csv('data/features/activity.csv')
performances = pd.read_csv('data/features/performances.csv')

In [283]:
activity[activity['activity_type']=='exam'] # this is weird --> exam as activity and most of those have time spent = 0

,activity_id,user_id,post_id,course_id,activity_type,activity_status,activity_started,activity_completed,activity_updated,domain,date_restored,times_valid,date,time_spent,time_in_minutes,time_truncated
50145,0,1286,1000000,5447,exam,1,2024-07-12 10:21:47,NaN,2024-07-12 10:21:47,essay,False,True,2024-07-12,0 days 00:00:00,0.000000,False
50146,1,1107,1000001,3301,exam,1,2024-07-12 14:53:34,NaN,2024-07-12 14:53:34,essay,False,True,2024-07-12,0 days 00:00:00,0.000000,False
50147,2,1286,1000002,5447,exam,1,2024-08-02 16:12:49,NaN,2024-08-02 16:12:49,essay,False,True,2024-08-02,0 days 00:00:00,0.000000,False
50148,3,1091,1000003,5447,exam,1,2024-08-05 11:17:34,NaN,2024-08-05 11:17:34,essay,False,True,2024-08-05,0 days 00:00:00,0.000000,False
50149,4,1091,1000004,5447,exam,1,2024-08-07 07:53:34,NaN,2024-08-07 07:53:34,essay,False,True,2024-08-07,0 days 00:00:00,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52739,627,2516,2000627,2115,exam,1,2025-01-04 13:42:05,2025-01-04 13:42:05,2025-01-04 13:42:05,text,False,True,2025-01-04,0 days 00:00:00,0.000000,False
52740,628,1629,2000628,5009,exam,1,2024-08-26 17:44:36,2024-08-26 18:25:07,2024-08-26 18:25:07,text,False,True,2024-08-26,0 days 00:40:31,40.516667,False
52741,629,2185,2000629,2115,exam,1,2024-10-30 14:02:31,2024-10-30 14:02:34,2024-10-30 14:02:34,text,False,True,2024-10-30,0 days 00:00:03,0.050000,False
52742,630,2516,2000630,5009,exam,1,2025-01-04 13:08:50,2025-01-04 13:11:53,2025-01-04 13:11:53,text,False,True,2025-01-04,0 days 00:03:03,3.050000,False


In [284]:
activity[activity['activity_type']=='exam'].time_in_minutes.value_counts()

time_in_minutes
0.000000     2225
75.000000      49
0.083333        7
0.133333        4
0.100000        3
             ... 
11.566667       1
65.633333       1
48.083333       1
48.500000       1
34.083333       1
Name: count, Length: 295, dtype: int64

In [285]:
activity.activity_type.value_counts()

activity_type
topic     30850
lesson    11348
quiz       6742
exam       2599
course      717
access      488
Name: count, dtype: int64

## First lets try to predict math exam results

In [286]:
user_days_math = user_days[user_days['domain']== 'math'].copy()
performances_math = performances[performances['domain']== 'math'].copy()
activity_math = activity[activity['domain']== 'math'].copy()

In [287]:
user_days_math

,user_id,date,type,user_day,number_of_activities,domain,activity_type,time_in_minutes
21,1,2025-02-13,activity,19,2,math,topic,16.116667
22,1,2025-02-15,activity,20,1,math,topic,30.000000
23,1,2025-02-16,activity,21,2,math,lesson,0.000000
24,1,2025-02-16,activity,21,2,math,topic,0.483333
25,1,2025-02-17,activity,22,3,math,lesson,0.000000
...,...,...,...,...,...,...,...,...
19821,4095,2024-10-30,both,14,8,math,topic,156.466667
19822,4095,2024-11-13,activity,15,5,math,topic,87.000000
19825,4095,2024-12-02,activity,17,3,math,topic,55.933333
19826,4095,2024-12-03,both,18,9,math,quiz,27.333333


In [288]:
activity_math

,activity_id,user_id,post_id,course_id,activity_type,activity_status,activity_started,activity_completed,activity_updated,domain,date_restored,times_valid,date,time_spent,time_in_minutes,time_truncated
0,1128,2533,42,42,course,0,2023-04-07 16:42:35,2023-04-07 17:35:15,2023-04-07 17:35:15,math,True,True,2023-04-07,0 days 00:52:40,52.666667,False
1,1129,2533,55,42,lesson,0,2023-04-07 16:42:35,NaN,2023-04-07 16:42:35,math,False,True,2023-04-07,0 days 00:00:00,0.000000,False
2,1130,2533,98,42,topic,1,2023-04-07 16:42:38,2023-04-07 16:43:58,2023-04-07 16:43:58,math,False,True,2023-04-07,0 days 00:01:20,1.333333,False
3,1131,2533,100,42,topic,1,2023-04-07 16:43:59,2023-04-07 16:46:13,2023-04-07 16:46:13,math,False,True,2023-04-07,0 days 00:02:14,2.233333,False
4,1132,2533,102,42,topic,1,2023-04-07 16:46:14,2023-04-07 16:46:27,2023-04-07 16:46:27,math,False,True,2023-04-07,0 days 00:00:13,0.216667,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50137,114742,955,647,42,topic,0,2025-03-07 06:36:32,NaN,2025-03-07 06:36:32,math,False,True,2025-03-07,0 days 00:00:00,0.000000,False
50138,114743,955,106,42,topic,0,2025-03-07 06:37:49,NaN,2025-03-07 06:37:49,math,False,True,2025-03-07,0 days 00:00:00,0.000000,False
50139,114744,955,104,42,topic,0,2025-03-07 06:37:50,NaN,2025-03-07 06:37:50,math,False,True,2025-03-07,0 days 00:00:00,0.000000,False
50140,114745,955,108,42,topic,0,2025-03-07 06:37:54,NaN,2025-03-07 06:37:54,math,False,True,2025-03-07,0 days 00:00:00,0.000000,False


In [289]:
activity_math.isnull().sum()

activity_id               0
user_id                   0
post_id                   0
course_id                 0
activity_type             0
activity_status           0
activity_started          0
activity_completed    12676
activity_updated          0
domain                    0
date_restored             0
times_valid               0
date                      0
time_spent                0
time_in_minutes           0
time_truncated            0
dtype: int64

In [240]:
performances_math

,user_id,domain,test_id,course,date,percentage,performance
9,6,math,42,3865,2024-11-23,25.00,-36.04
10,6,math,48,3865,2025-01-08,50.00,-1.92
11,6,math,49,3865,2025-01-08,66.67,21.23
12,6,math,50,3865,2025-02-04,54.55,19.57
13,6,math,54,3865,2024-11-23,14.29,-47.71
...,...,...,...,...,...,...,...
4826,4095,math,49,3865,2024-10-18,0.00,-45.44
4827,4095,math,50,3865,2024-10-18,0.00,-34.98
4828,4095,math,51,3865,2024-09-24,20.00,-22.98
4829,4095,math,53,3865,2024-09-24,0.00,-56.46


In [241]:
activity_math.dropna(inplace=True) # not sure what to do here, drops almost 50% of the rows but would those rows would fuck up the time spent on activity feature as it was 0 for all of them

In [242]:
activity_math.activity_type.value_counts()

activity_type
topic     6699
quiz      6278
lesson     143
access      74
course      68
Name: count, dtype: int64

## Feature extraction

In [260]:
# Rolling window for recent activity
rolling_window_days = 10

# Convert the 'date' columns to datetime
activity_math['date'] = pd.to_datetime(activity_math['date'])
performances_math['date'] = pd.to_datetime(performances_math['date'])

def compute_all_features_for_exam(exam_row, user_activities, user_exams, window_days=rolling_window_days):

    exam_date = exam_row['date']

    # Include activities up to and including exam_date
    previous_activities = user_activities[user_activities['date'] <= exam_date].copy() # need to modify to have the exact date and time in the performance df, because here activities that happend after the exam on the same day will be accounted for.

    # Rolling window (activities in the last N days, including exam day)
    window_start = exam_date - pd.Timedelta(days=window_days)
    rolling_activities = previous_activities[previous_activities['date'] >= window_start].copy()

    features = {}

    # Recent average time per activity (rolling window)
    total_time_rolling = rolling_activities['time_in_minutes'].sum()
    count_rolling = len(rolling_activities)
    features['recent_avg_time_per_activity'] = total_time_rolling / count_rolling if count_rolling > 0 else np.nan

    # Number of days since last activity
    if not previous_activities.empty:
        last_activity_date = previous_activities['date'].max()
        features['days_since_last_activity'] = (exam_date - last_activity_date).days
    else:
        features['days_since_last_activity'] = np.nan

    # Total time spent on activities before the exam
    features['total_time_spent_on_activity_before_exam'] = previous_activities['time_in_minutes'].sum() if not previous_activities.empty else np.nan

    # Average percentage on past exams
    previous_exams = user_exams[user_exams['date'] < exam_date]
    features['average_percentage_past_exams'] = previous_exams['percentage'].mean() if not previous_exams.empty else np.nan

    # Usage Frequency: Average activities per day in rolling window & Active days ratio
    features['avg_activities_per_day_recent'] = count_rolling / window_days if window_days > 0 else np.nan
    if not rolling_activities.empty:
        distinct_days = rolling_activities['date'].dt.normalize().nunique()
    else:
        distinct_days = 0
    features['active_days_ratio_recent'] = distinct_days / window_days if window_days > 0 else np.nan

    # Activity diversity (rolling window)
    features['diversity_recent'] = rolling_activities['activity_type'].nunique() if not rolling_activities.empty else np.nan


    return pd.Series(features)

# Loop over each exam (grouped by user) in performances_math and compute all features.
features_list = []

for user_id, user_exams in performances_math.groupby('user_id'):
    # Get corresponding activities for the user from activity_math and sort by date
    user_activities = activity_math[activity_math['user_id'] == user_id].sort_values('date')
    user_exams_sorted = user_exams.sort_values('date')

    for exam_index, exam_row in user_exams_sorted.iterrows():
        feats = compute_all_features_for_exam(exam_row, user_activities, user_exams_sorted, rolling_window_days)
        feats['exam_index'] = exam_index
        features_list.append(feats)

# Output df
features_df = pd.DataFrame(features_list).set_index('exam_index')
performances_math_features = performances_math.join(features_df, how='left')

In [261]:
performances_math_features

,user_id,domain,test_id,course,date,percentage,performance,recent_avg_time_per_activity,days_since_last_activity,total_time_spent_on_activity_before_exam,average_percentage_past_exams,avg_activities_per_day_recent,active_days_ratio_recent,diversity_recent
9.0,6,math,42,3865,2024-11-23,25.00,-36.04,22.701282,0.0,299.533333,NaN,1.3,0.2,2.0
10.0,6,math,48,3865,2025-01-08,50.00,-1.92,28.891667,0.0,472.883333,29.822500,0.6,0.2,2.0
11.0,6,math,49,3865,2025-01-08,66.67,21.23,28.891667,0.0,472.883333,29.822500,0.6,0.2,2.0
12.0,6,math,50,3865,2025-02-04,54.55,19.57,89.533333,0.0,597.816667,39.326667,0.1,0.1,1.0
13.0,6,math,54,3865,2024-11-23,14.29,-47.71,22.701282,0.0,299.533333,NaN,1.3,0.2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4826.0,4095,math,49,3865,2024-10-18,0.00,-45.44,91.672222,0.0,489.600000,10.833333,0.3,0.1,1.0
4827.0,4095,math,50,3865,2024-10-18,0.00,-34.98,91.672222,0.0,489.600000,10.833333,0.3,0.1,1.0
4828.0,4095,math,51,3865,2024-09-24,20.00,-22.98,15.388095,0.0,197.716667,NaN,0.7,0.2,2.0
4829.0,4095,math,53,3865,2024-09-24,0.00,-56.46,15.388095,0.0,197.716667,NaN,0.7,0.2,2.0


 Here not sure how we should handle the average percentage on last exams when it's the first exam a student takes do we take the average ?


In [245]:
from sklearn.preprocessing import StandardScaler


# scaling the columns
columns_to_scale = ['recent_avg_time_per_activity', 'days_since_last_activity', 'total_time_spent_on_activity_before_exam','average_percentage_past_exams','avg_activities_per_day_recent','diversity_recent']


scaler = StandardScaler()
scaled_values = scaler.fit_transform(performances_math_features[columns_to_scale])
scaled_df = pd.DataFrame(scaled_values, columns=columns_to_scale, index=performances_math_features.index)
remaining_df = performances_math_features.drop(columns=columns_to_scale)
final_df = pd.concat([scaled_df, remaining_df], axis=1)

In [246]:
final_df.isnull().sum()

recent_avg_time_per_activity                  8
days_since_last_activity                      2
total_time_spent_on_activity_before_exam      2
average_percentage_past_exams               777
avg_activities_per_day_recent                 0
diversity_recent                              8
user_id                                       0
domain                                        0
test_id                                       0
course                                        0
date                                          0
percentage                                    0
performance                                   0
active_days_ratio_recent                      0
dtype: int64

In [247]:
final_df.dropna(inplace=True)

In [248]:
import statsmodels.formula.api as smf

# Linear Regression Model
mod = smf.ols(formula='performance ~  recent_avg_time_per_activity + days_since_last_activity + total_time_spent_on_activity_before_exam + average_percentage_past_exams + avg_activities_per_day_recent + diversity_recent', data=final_df)

# Fit the model
res = mod.fit()

# Print regression results summary
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            performance   R-squared:                       0.240
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     158.7
Date:                Thu, 17 Apr 2025   Prob (F-statistic):          1.20e-175
Time:                        09:44:27   Log-Likelihood:                -14026.
No. Observations:                3028   AIC:                         2.807e+04
Df Residuals:                    3021   BIC:                         2.811e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

## Let's try with the essay results

In [249]:
performances_essay = performances[performances['domain']== 'essay'].copy()
activity_essay= activity[activity['domain']== 'essay'].copy()

In [250]:
activity_essay.isnull().sum() # big issue with the activity completed comlumn, way too many nans.

activity_id              0
user_id                  0
post_id                  0
course_id                0
activity_type            0
activity_status          0
activity_started         0
activity_completed    9550
activity_updated         0
domain                   0
date_restored            0
times_valid              0
date                     0
time_spent               0
time_in_minutes          0
time_truncated           0
dtype: int64

In [251]:
activity_essay.dropna(inplace=True)

In [252]:
activity_essay

,activity_id,user_id,post_id,course_id,activity_type,activity_status,activity_started,activity_completed,activity_updated,domain,date_restored,times_valid,date,time_spent,time_in_minutes,time_truncated
241,23817,2560,3301,3301,access,0,2023-11-26 08:58:42,2023-11-26 08:58:49,2023-11-26 08:58:49,essay,True,True,2023-11-26,0 days 00:00:07,0.116667,False
258,24116,2574,5447,5447,access,0,2023-11-27 13:33:10,2023-11-27 13:33:18,2023-11-27 13:33:18,essay,True,True,2023-11-27,0 days 00:00:08,0.133333,False
263,24131,2574,5466,5447,topic,1,2023-11-27 13:52:33,2023-11-27 15:54:13,2023-11-27 15:54:13,essay,False,True,2023-11-27,0 days 00:30:00,30.000000,True
264,24132,2574,5467,5447,topic,1,2023-11-27 13:52:37,2023-11-27 15:53:51,2023-11-27 15:53:51,essay,False,True,2023-11-27,0 days 00:30:00,30.000000,True
265,24133,2574,5469,5447,topic,1,2023-11-27 13:52:39,2023-11-27 15:50:05,2023-11-27 15:50:05,essay,False,True,2023-11-27,0 days 00:30:00,30.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49818,114296,944,15622,5447,quiz,1,2025-03-03 12:17:09,2025-03-03 12:21:15,2025-03-03 12:21:15,essay,False,True,2025-03-03,0 days 00:04:06,4.100000,False
50006,114564,959,15595,5447,topic,1,2025-03-05 16:31:00,2025-03-05 16:31:06,2025-03-05 16:31:06,essay,False,True,2025-03-05,0 days 00:00:06,0.100000,False
50056,114616,922,15727,5447,topic,1,2025-03-05 18:20:23,2025-03-05 18:20:43,2025-03-05 18:20:43,essay,False,True,2025-03-05,0 days 00:00:20,0.333333,False
50116,114704,955,5447,5447,course,0,2025-03-06 18:00:55,2025-03-06 18:27:06,2025-03-06 18:27:06,essay,True,True,2025-03-06,0 days 00:26:11,26.183333,False


In [253]:
performances_essay.isnull().sum()

user_id        0
domain         0
test_id        0
course         0
date           0
percentage     0
performance    0
dtype: int64

In [262]:
# Convert the date columns  to datetime
activity_essay['date'] = pd.to_datetime(activity_essay['date'])
performances_essay['date'] = pd.to_datetime(performances_essay['date'])

# Loop over each exam (grouped by user) in performances_essay and compute all features.
features_list = []

for user_id, user_exams in performances_essay.groupby('user_id'):
    # Get corresponding activities for the user from activity_essay and sort by date
    user_activities = activity_essay[activity_essay['user_id'] == user_id].sort_values('date')
    user_exams_sorted = user_exams.sort_values('date')

    for exam_index, exam_row in user_exams_sorted.iterrows():
        feats = compute_all_features_for_exam(exam_row, user_activities, user_exams_sorted, rolling_window_days)
        feats['exam_index'] = exam_index
        features_list.append(feats)

# Output df
features_df = pd.DataFrame(features_list).set_index('exam_index')
performances_essay_features = performances_essay.join(features_df, how='left')

In [265]:
performances_essay_features

,user_id,domain,test_id,course,date,percentage,performance,recent_avg_time_per_activity,days_since_last_activity,total_time_spent_on_activity_before_exam,average_percentage_past_exams,avg_activities_per_day_recent,active_days_ratio_recent,diversity_recent
0.0,1,essay,eroerterung,3301,2024-11-29,63.000000,-7.090000,NaN,27.0,4.983333,55.294118,0.0,0.0,NaN
1.0,1,essay,erzaehlung,5447,2024-10-26,55.294118,-8.075882,NaN,NaN,NaN,NaN,0.0,0.0,NaN
2.0,4,essay,eroerterung,3301,2024-11-21,66.000000,-4.090000,11.233333,10.0,22.466667,71.000000,0.2,0.1,1.0
3.0,4,essay,erzaehlung,3301,2024-11-07,71.000000,3.390000,NaN,NaN,NaN,NaN,0.0,0.0,NaN
4.0,5,essay,erzaehlung,5447,2024-10-26,44.705882,-18.664118,NaN,NaN,NaN,NaN,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4781.0,4087,essay,eroerterung,3301,2024-12-17,64.000000,-6.090000,NaN,97.0,34.566667,71.000000,0.0,0.0,NaN
4782.0,4087,essay,erzaehlung,3301,2024-11-03,71.000000,3.390000,NaN,53.0,34.566667,NaN,0.0,0.0,NaN
4806.0,4089,essay,erzaehlung,5447,2024-11-03,50.588235,-12.781765,NaN,49.0,112.650000,NaN,0.0,0.0,NaN
4807.0,4092,essay,beschreibung,3301,2024-12-24,69.000000,-0.960000,17.483333,0.0,353.933333,NaN,1.6,0.1,3.0


In [264]:
performances_essay_features.isnull().sum() # too many nans in the features for essay.

user_id                                       0
domain                                        0
test_id                                       0
course                                        0
date                                          0
percentage                                    0
performance                                   0
recent_avg_time_per_activity                434
days_since_last_activity                    177
total_time_spent_on_activity_before_exam    177
average_percentage_past_exams               328
avg_activities_per_day_recent                 0
active_days_ratio_recent                      0
diversity_recent                            434
dtype: int64

In [266]:
performances_essay_features.dropna(inplace=True)

In [267]:
performances_essay_features # only 65 rows left

,user_id,domain,test_id,course,date,percentage,performance,recent_avg_time_per_activity,days_since_last_activity,total_time_spent_on_activity_before_exam,average_percentage_past_exams,avg_activities_per_day_recent,active_days_ratio_recent,diversity_recent
2.0,4,essay,eroerterung,3301,2024-11-21,66.000000,-4.090000,11.233333,10.0,22.466667,71.000000,0.2,0.1,1.0
5.0,6,essay,bericht,3301,2024-11-05,60.000000,-11.480000,59.479167,10.0,475.833333,52.000000,0.8,0.1,3.0
54.0,17,essay,beschreibung,5447,2024-11-17,27.058824,-43.721176,2.166667,5.0,2.166667,30.588235,0.1,0.1,1.0
189.0,81,essay,eroerterung,3301,2024-12-30,69.000000,-1.090000,3.756667,0.0,212.300000,62.000000,0.5,0.1,1.0
321.0,224,essay,beschreibung,3301,2024-11-08,65.000000,-4.960000,7.938889,7.0,23.816667,57.000000,0.3,0.1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3711.0,3963,essay,beschreibung,5447,2024-12-02,70.588235,-0.191765,20.033333,7.0,20.033333,60.000000,0.1,0.1,1.0
3713.0,3964,essay,bericht,5447,2024-11-30,77.647059,10.487059,19.566667,8.0,173.133333,67.058824,0.1,0.1,1.0
3769.0,3977,essay,bericht,5447,2024-12-04,71.764706,4.604706,3.711111,8.0,32.233333,92.941176,0.3,0.1,1.0
3824.0,3999,essay,erzaehlung,3301,2024-12-08,77.000000,9.390000,3.753333,8.0,22.866667,83.000000,0.5,0.1,1.0


In [268]:
# scale the df

columns_to_scale = ['recent_avg_time_per_activity', 'days_since_last_activity', 'total_time_spent_on_activity_before_exam','average_percentage_past_exams','avg_activities_per_day_recent','diversity_recent']


scaler = StandardScaler()
scaled_values = scaler.fit_transform(performances_essay_features[columns_to_scale])
scaled_df = pd.DataFrame(scaled_values, columns=columns_to_scale, index=performances_essay_features.index)

remaining_df = performances_essay_features.drop(columns=columns_to_scale)

final_df = pd.concat([scaled_df, remaining_df], axis=1)

In [269]:
# Linear Regression Model
mod = smf.ols(formula='performance ~  recent_avg_time_per_activity + days_since_last_activity + total_time_spent_on_activity_before_exam + average_percentage_past_exams + avg_activities_per_day_recent + diversity_recent', data=final_df)

# Fit the model
res = mod.fit()

# Print regression results summary
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            performance   R-squared:                       0.349
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     5.190
Date:                Thu, 17 Apr 2025   Prob (F-statistic):           0.000248
Time:                        09:57:09   Log-Likelihood:                -249.48
No. Observations:                  65   AIC:                             513.0
Df Residuals:                      58   BIC:                             528.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

## Let's try with the text results now


In [270]:
performances_text = performances[performances['domain']== 'text'].copy()
activity_text= activity[activity['domain']== 'text'].copy()

In [271]:
activity_text.isnull().sum() # issue with the activity completed comlumn, way too many nans. Idk how we should handle those.

activity_id              0
user_id                  0
post_id                  0
course_id                0
activity_type            0
activity_status          0
activity_started         0
activity_completed    7666
activity_updated         0
domain                   0
date_restored            0
times_valid              0
date                     0
time_spent               0
time_in_minutes          0
time_truncated           0
dtype: int64

In [272]:
activity_text

,activity_id,user_id,post_id,course_id,activity_type,activity_status,activity_started,activity_completed,activity_updated,domain,date_restored,times_valid,date,time_spent,time_in_minutes,time_truncated
18,1207,2533,2115,2115,access,0,2023-05-02 08:49:46,2023-05-02 08:49:46,2023-05-02 08:49:46,text,True,True,2023-05-02,0 days 00:00:00,0.000000,False
19,1208,2533,2115,2115,course,0,2023-05-02 08:50:00,2023-05-02 08:50:16,2023-05-02 08:50:16,text,True,True,2023-05-02,0 days 00:00:16,0.266667,False
20,1209,2533,2150,2115,lesson,0,2023-05-02 08:50:00,NaN,2023-05-02 08:50:00,text,False,True,2023-05-02,0 days 00:00:00,0.000000,False
21,1210,2533,2117,2115,topic,1,2023-05-02 08:50:00,2023-05-02 08:50:16,2023-05-02 08:50:16,text,False,True,2023-05-02,0 days 00:00:16,0.266667,False
22,1211,2533,2152,2115,topic,0,2023-05-02 08:50:16,NaN,2023-05-02 08:50:16,text,False,True,2023-05-02,0 days 00:00:00,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52739,627,2516,2000627,2115,exam,1,2025-01-04 13:42:05,2025-01-04 13:42:05,2025-01-04 13:42:05,text,False,True,2025-01-04,0 days 00:00:00,0.000000,False
52740,628,1629,2000628,5009,exam,1,2024-08-26 17:44:36,2024-08-26 18:25:07,2024-08-26 18:25:07,text,False,True,2024-08-26,0 days 00:40:31,40.516667,False
52741,629,2185,2000629,2115,exam,1,2024-10-30 14:02:31,2024-10-30 14:02:34,2024-10-30 14:02:34,text,False,True,2024-10-30,0 days 00:00:03,0.050000,False
52742,630,2516,2000630,5009,exam,1,2025-01-04 13:08:50,2025-01-04 13:11:53,2025-01-04 13:11:53,text,False,True,2025-01-04,0 days 00:03:03,3.050000,False


In [273]:
activity_text.dropna(inplace=True)

In [274]:
# Convert the date columns to datetime
activity_text['date'] = pd.to_datetime(activity_text['date'])
performances_text['date'] = pd.to_datetime(performances_text['date'])

# Loop over each exam (grouped by user) in performances_text and compute all features.
features_list = []

for user_id, user_exams in performances_text.groupby('user_id'):
    # Get corresponding activities for the user from activity_text and sort by date
    user_activities = activity_text[activity_text['user_id'] == user_id].sort_values('date')
    user_exams_sorted = user_exams.sort_values('date')

    for exam_index, exam_row in user_exams_sorted.iterrows():
        feats = compute_all_features_for_exam(exam_row, user_activities, user_exams_sorted, rolling_window_days)
        feats['exam_index'] = exam_index
        features_list.append(feats)

# Output df
features_df = pd.DataFrame(features_list).set_index('exam_index')
performances_text_features = performances_text.join(features_df, how='left')

In [275]:
performances_text_features

,user_id,domain,test_id,course,date,percentage,performance,recent_avg_time_per_activity,days_since_last_activity,total_time_spent_on_activity_before_exam,average_percentage_past_exams,avg_activities_per_day_recent,active_days_ratio_recent,diversity_recent
192.0,81,text,1,2115,2025-01-04,0.000000,0.000000,NaN,11.0,6.100000,NaN,0.0,0.0,NaN
193.0,81,text,10,5009,2025-01-04,0.000000,-48.710000,NaN,11.0,6.100000,NaN,0.0,0.0,NaN
205.0,90,text,1,2115,2024-12-15,0.000000,0.000000,14.372222,0.0,43.116667,NaN,0.3,0.2,2.0
206.0,90,text,3,2115,2024-12-17,76.086957,29.136957,9.577778,0.0,57.466667,31.000000,0.6,0.3,2.0
207.0,90,text,4,2115,2024-12-15,62.000000,18.720000,14.372222,0.0,43.116667,NaN,0.3,0.2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4831.0,4095,text,1,2115,2024-08-21,0.000000,0.000000,NaN,NaN,NaN,NaN,0.0,0.0,NaN
4832.0,4095,text,10,5009,2024-08-21,32.727273,-15.982727,NaN,NaN,NaN,NaN,0.0,0.0,NaN
4833.0,4095,text,13,5009,2024-10-29,40.298507,-12.121493,NaN,NaN,NaN,16.363636,0.0,0.0,NaN
4834.0,4095,text,14,5009,2024-11-25,52.702703,-6.777297,NaN,NaN,NaN,24.341927,0.0,0.0,NaN


In [276]:
performances_text_features.isnull().sum() # too many nans

user_id                                       0
domain                                        0
test_id                                       0
course                                        0
date                                          0
percentage                                    0
performance                                   0
recent_avg_time_per_activity                363
days_since_last_activity                    304
total_time_spent_on_activity_before_exam    304
average_percentage_past_exams               219
avg_activities_per_day_recent                 0
active_days_ratio_recent                      0
diversity_recent                            363
dtype: int64

In [277]:
performances_text_features.dropna(inplace=True)

In [278]:
performances_text_features

,user_id,domain,test_id,course,date,percentage,performance,recent_avg_time_per_activity,days_since_last_activity,total_time_spent_on_activity_before_exam,average_percentage_past_exams,avg_activities_per_day_recent,active_days_ratio_recent,diversity_recent
206.0,90,text,3,2115,2024-12-17,76.086957,29.136957,9.577778,0.0,57.466667,31.000000,0.6,0.3,2.0
208.0,90,text,5,2115,2024-12-21,59.615385,2.835385,8.629630,0.0,77.666667,46.028986,0.9,0.4,2.0
209.0,90,text,6,2115,2024-12-23,61.224490,13.434490,8.629630,2.0,77.666667,49.425585,0.9,0.4,2.0
210.0,90,text,7,2115,2024-12-28,59.090909,-3.179091,6.733333,7.0,77.666667,51.785366,0.3,0.1,1.0
211.0,90,text,8,2115,2024-12-29,51.515152,6.445152,6.733333,8.0,77.666667,53.002957,0.3,0.1,1.0
567.0,265,text,6,2115,2024-11-21,30.000000,-17.790000,0.200000,2.0,0.200000,14.583333,0.1,0.1,1.0
601.0,269,text,6,2115,2024-11-23,36.956522,-10.833478,0.783333,6.0,0.783333,25.961538,0.1,0.1,1.0
1702.0,489,text,8,2115,2025-01-05,42.105263,-2.964737,0.194737,0.0,3.700000,39.620915,1.9,0.1,2.0
1719.0,490,text,4,2115,2024-11-27,34.000000,-9.280000,7.605556,0.0,73.050000,30.641026,0.3,0.2,1.0
1722.0,490,text,7,2115,2024-12-10,62.000000,-0.270000,24.422222,0.0,146.316667,31.480769,0.3,0.2,1.0


In [279]:
# Scale the columns
columns_to_scale = ['recent_avg_time_per_activity', 'days_since_last_activity', 'total_time_spent_on_activity_before_exam','average_percentage_past_exams','avg_activities_per_day_recent','diversity_recent']

scaler = StandardScaler()
scaled_values = scaler.fit_transform(performances_text_features[columns_to_scale])
scaled_df = pd.DataFrame(scaled_values, columns=columns_to_scale, index=performances_text_features.index)

remaining_df = performances_text_features.drop(columns=columns_to_scale)

final_df = pd.concat([scaled_df, remaining_df], axis=1)

In [280]:
final_df # only 25 rows left, not really relevant

,recent_avg_time_per_activity,days_since_last_activity,total_time_spent_on_activity_before_exam,average_percentage_past_exams,avg_activities_per_day_recent,diversity_recent,user_id,domain,test_id,course,date,percentage,performance,active_days_ratio_recent
206.0,0.388782,-0.812920,-0.323634,-0.532226,0.256319,2.0,90,text,3,2115,2024-12-17,76.086957,29.136957,0.3
208.0,0.231016,-0.812920,-0.064234,0.570346,0.968316,2.0,90,text,5,2115,2024-12-21,59.615385,2.835385,0.4
209.0,0.231016,-0.167745,-0.064234,0.819531,0.968316,2.0,90,text,6,2115,2024-12-23,61.224490,13.434490,0.4
210.0,-0.084516,1.445191,-0.064234,0.992652,-0.455678,-0.5,90,text,7,2115,2024-12-28,59.090909,-3.179091,0.1
211.0,-0.084516,1.767779,-0.064234,1.081978,-0.455678,-0.5,90,text,8,2115,2024-12-29,51.515152,6.445152,0.1
567.0,-1.171622,-0.167745,-1.059031,-1.736604,-0.930342,-0.5,265,text,6,2115,2024-11-21,30.000000,-17.790000,0.1
601.0,-1.074559,1.122604,-1.051540,-0.901863,-0.930342,-0.5,269,text,6,2115,2024-11-23,36.956522,-10.833478,0.1
1702.0,-1.172498,-0.812920,-1.014085,0.100230,3.341638,2.0,489,text,8,2115,2025-01-05,42.105263,-2.964737,0.1
1719.0,0.060616,-0.812920,-0.123519,-0.558562,-0.455678,-0.5,490,text,4,2115,2024-11-27,34.000000,-9.280000,0.2
1722.0,2.858805,-0.812920,0.817343,-0.496956,-0.455678,-0.5,490,text,7,2115,2024-12-10,62.000000,-0.270000,0.2


In [281]:
# Linear Regression Model
mod = smf.ols(formula='performance ~  recent_avg_time_per_activity + days_since_last_activity + total_time_spent_on_activity_before_exam + average_percentage_past_exams + avg_activities_per_day_recent + diversity_recent', data=final_df)

# Fit the model
res = mod.fit()

# Print regression results summary
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            performance   R-squared:                       0.175
Model:                            OLS   Adj. R-squared:                 -0.100
Method:                 Least Squares   F-statistic:                    0.6372
Date:                Thu, 17 Apr 2025   Prob (F-statistic):              0.699
Time:                        10:11:37   Log-Likelihood:                -98.268
No. Observations:                  25   AIC:                             210.5
Df Residuals:                      18   BIC:                             219.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------